In [1]:
import pandas as pd


SV_df = pd.read_csv("data/NOW_preprocessed/total_SV_df.csv")
VO_df = pd.read_csv("data/NOW_preprocessed/total_VO_df.csv")

print(len(SV_df))
print(len(VO_df))

1085437
570373


## Bringing SV, VO columns to a preprocessed format ##

In [2]:
# checking how many NAs there are

print(SV_df['SVs'].isna().sum()) 
print(VO_df['VOs'].isna().sum())

# removing the NAs

SV_df_nona = SV_df[SV_df['SVs'].notna()]
VO_df_nona = VO_df[VO_df['VOs'].notna()]


print(SV_df_nona['SVs'].isna().sum()) 
print(VO_df_nona['VOs'].isna().sum())

print(len(SV_df_nona))
print(len(VO_df_nona))

7
202
0
0
1085430
570171


In [3]:
# taking the S, V, O columns out of list format, making it into string:

import re

def text_cleaner(text):
    text_cleaned = re.sub(r",", "", text) 
    return text_cleaned


# making the list column into string on the SV df

SV_df_nona['S_string'] = SV_df_nona['S'].apply(lambda x: x[1:-1])
SV_df_nona['S_string'] = SV_df_nona.apply(lambda row: text_cleaner(row["S_string"]), axis = 1)
SV_df_nona['V_string'] = SV_df_nona['V'].apply(lambda x: x[1:-1])

SV_df_nona


# making the list column into string on the VO df

VO_df_nona['V_string'] = VO_df_nona['V'].apply(lambda x: x[1:-1])
VO_df_nona['O_string'] = VO_df_nona['O'].apply(lambda x: x[1:-1])
VO_df_nona['O_string'] = VO_df_nona.apply(lambda row: text_cleaner(row["O_string"]), axis = 1)

VO_df_nona


<ipython-input-3-252833377575>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SV_df_nona['S_string'] = SV_df_nona['S'].apply(lambda x: x[1:-1])
<ipython-input-3-252833377575>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SV_df_nona['S_string'] = SV_df_nona.apply(lambda row: text_cleaner(row["S_string"]), axis = 1)
<ipython-input-3-252833377575>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,ID,Date,News_source_name,VOs,V,O,V_string,O_string
0,31930103,20-07-01,npr.org,"([seen], [[John, Hickenlooper], [Colorado, Sen...",[seen],"[[John, Hickenlooper], [Colorado, Senate, Prim...",seen,[John Hickenlooper] [Colorado Senate Primary]
1,31930103,20-07-01,npr.org,"([made], [missteps])",[made],[missteps],made,missteps
2,31930103,20-07-01,npr.org,"([hide], [caption])",[hide],[caption],hide,caption
3,31930103,20-07-01,npr.org,"([considered], [John, Hickenlooper])",[considered],"[John, Hickenlooper]",considered,John Hickenlooper
4,31930103,20-07-01,npr.org,"([made], [missteps])",[made],[missteps],made,missteps
...,...,...,...,...,...,...,...,...
570368,85309194,20-06-30,YAHOO!,"([posted], [gains])",[posted],[gains],posted,gains
570369,85309194,20-06-30,YAHOO!,"([led], [way])",[led],[way],led,way
570370,85309194,20-06-30,YAHOO!,"([averaged], [viewers])",[averaged],[viewers],averaged,viewers
570371,85309194,20-06-30,YAHOO!,"([drew], [viewers])",[drew],[viewers],drew,viewers


### filtering for S and O that includes BLM relevant terms. Worthy revisions: extend term list, save to csv. #### Important to note: if term list fitlers are changed, the news sources have to be reassessed. Additional news sources might be inlcuded in the exported csv files, which have to be checked for bias ratings

In [4]:
# filtering to see how many BLM relevant Subjects there are in SV df
# adding a numeric code, representing the following: -1 = BLM side (against authority - liberal), 1 = anti-BLM side (for authority - conservative), 0 = shoudl trigger disapproval by both sides, e.g. Zimmerman, brutality, Chauvin

SV_df_nona_breonna_S = SV_df_nona[SV_df_nona['S_string'].str.contains("Breonna", case = False)]
SV_df_nona_breonna_S["Side"] = "-1"
SV_df_nona_floyd_S = SV_df_nona[SV_df_nona['S_string'].str.contains("Floyd", case = False)]
SV_df_nona_floyd_S["Side"] = "-1"
SV_df_nona_trayvon_S = SV_df_nona[SV_df_nona['S_string'].str.contains("Trayvon", case = False)]
SV_df_nona_trayvon_S["Side"] = "-1"
SV_df_nona_movement_S = SV_df_nona[SV_df_nona['S_string'].str.contains("movement", case = False)]
SV_df_nona_movement_S["Side"] = "-1"
SV_df_nona_blacklivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("Black Lives Matter", case = False)]
SV_df_nona_blacklivesmatter_S["Side"] = "-1"
SV_df_nona_blm_S = SV_df_nona[SV_df_nona['S_string'].str.contains("BLM", case = False)]
SV_df_nona_blm_S["Side"] = "-1"
SV_df_nona_htblacklivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("#BlackLivesMatter", case = False)]
SV_df_nona_htblacklivesmatter_S["Side"] = "-1"
SV_df_nona_protester_S = SV_df_nona[SV_df_nona['S_string'].str.contains("protester", case = False)]
SV_df_nona_protester_S["Side"] = "-1"
SV_df_nona_demonstrator_S = SV_df_nona[SV_df_nona['S_string'].str.contains("demonstrator", case = False)]
SV_df_nona_demonstrator_S["Side"] = "-1"
SV_df_nona_nationalguard_S = SV_df_nona[SV_df_nona['S_string'].str.contains("National Guard", case = False)]
SV_df_nona_nationalguard_S["Side"] = "1"
SV_df_nona_bluelivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("Blue Lives Matter", case = False)]
SV_df_nona_bluelivesmatter_S["Side"] = "1"
SV_df_nona_htbluelivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("#BlueLivesMatter", case = False)]
SV_df_nona_htbluelivesmatter_S["Side"] = "1"
SV_df_nona_alllivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("All Lives Matter", case = False)]
SV_df_nona_alllivesmatter_S["Side"] = "1"
SV_df_nona_htalllivesmatter_S = SV_df_nona[SV_df_nona['S_string'].str.contains("#AllLivesMatter", case = False)]
SV_df_nona_htalllivesmatter_S["Side"] = "1"
SV_df_nona_police_S = SV_df_nona[SV_df_nona['S_string'].str.contains("police", case = False)]
SV_df_nona_police_S["Side"] = "1"
SV_df_nona_government_S = SV_df_nona[SV_df_nona['S_string'].str.contains("government", case = False)]
SV_df_nona_government_S["Side"] = "1"


SV_df_nona_terms = pd.concat([SV_df_nona_breonna_S,
                               SV_df_nona_floyd_S,
                               SV_df_nona_trayvon_S,
                               SV_df_nona_movement_S,
                               SV_df_nona_blacklivesmatter_S,
                               SV_df_nona_blm_S,
                               SV_df_nona_htblacklivesmatter_S,
                               SV_df_nona_protester_S,
                               SV_df_nona_demonstrator_S,
                               SV_df_nona_nationalguard_S,
                               SV_df_nona_bluelivesmatter_S,
                               SV_df_nona_htbluelivesmatter_S,
                               SV_df_nona_alllivesmatter_S,
                               SV_df_nona_htalllivesmatter_S,
                               SV_df_nona_police_S,
                               SV_df_nona_government_S
                              ],
                                 ignore_index=True)

SV_df_nona_terms

<ipython-input-4-c8b61c8aee2c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SV_df_nona_breonna_S["Side"] = "-1"
<ipython-input-4-c8b61c8aee2c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SV_df_nona_floyd_S["Side"] = "-1"
<ipython-input-4-c8b61c8aee2c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

,ID,Date,News_source_name,SVs,S,V,S_string,V_string,Side
0,85330111,20-07-03,Washington Post,"([Breonna, Taylor], [sleeping])","[Breonna, Taylor]",[sleeping],Breonna Taylor,sleeping,-1
1,85395800,20-07-11,YAHOO!,"([Breonna, Taylor], [used])","[Breonna, Taylor]",[used],Breonna Taylor,used,-1
2,85543408,20-07-30,NBC News,"([Breonna, Taylor], [cover])","[Breonna, Taylor]",[cover],Breonna Taylor,cover,-1
3,85068543,20-06-01,Milwaukee Journal Sentinel,"([Breonna, Taylor], [wanted])","[Breonna, Taylor]",[wanted],Breonna Taylor,wanted,-1
4,85101644,20-06-05,E News,"([Breonna], [celebrating])",[Breonna],[celebrating],Breonna,celebrating,-1
...,...,...,...,...,...,...,...,...,...
34440,85288194,20-06-28,Reason,"([government], [wants])",[government],[wants],government,wants,1
34441,85291798,20-06-28,NaN,"([government], [offers])",[government],[offers],government,offers,1
34442,85294490,20-06-29,Reason,"([governments], [approved])",[governments],[approved],governments,approved,1
34443,85294490,20-06-29,Reason,"([government], [make])",[government],[make],government,make,1


In [5]:
# filtering to see how many BLM relevant Objects there are in VO df
# adding a numeric code, representing the following: -1 = BLM side (against authority - liberal), 1 = anti-BLM side (for authority - conservative), 0 = shoudl trigger disapproval by both sides, e.g. Zimmerman, brutality, Chauvin

VO_df_nona_breonna_O = VO_df_nona[VO_df_nona['O_string'].str.contains("Breonna", case = False)]
VO_df_nona_breonna_O["Side"] = "-1"
VO_df_nona_floyd_O = VO_df_nona[VO_df_nona['O_string'].str.contains("Floyd", case = False)]
VO_df_nona_floyd_O["Side"] = "-1"
VO_df_nona_trayvon_O = VO_df_nona[VO_df_nona['O_string'].str.contains("Trayvon", case = False)]
VO_df_nona_trayvon_O["Side"] = "-1"
VO_df_nona_movement_O = VO_df_nona[VO_df_nona['O_string'].str.contains("movement", case = False)]
VO_df_nona_movement_O["Side"] = "-1"
VO_df_nona_blacklivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("Black Lives Matter", case = False)]
VO_df_nona_blacklivesmatter_O["Side"] = "-1"
VO_df_nona_blm_O = VO_df_nona[VO_df_nona['O_string'].str.contains("BLM", case = False)]
VO_df_nona_blm_O["Side"] = "-1"
VO_df_nona_htblacklivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("#BlackLivesMatter", case = False)]
VO_df_nona_htblacklivesmatter_O["Side"] = "-1"
VO_df_nona_protester_O = VO_df_nona[VO_df_nona['O_string'].str.contains("protester", case = False)]
VO_df_nona_protester_O["Side"] = "-1"
VO_df_nona_demonstrator_O = VO_df_nona[VO_df_nona['O_string'].str.contains("demonstrator", case = False)]
VO_df_nona_demonstrator_O["Side"] = "-1"
VO_df_nona_nationalguard_O = VO_df_nona[VO_df_nona['O_string'].str.contains("National Guard", case = False)]
VO_df_nona_nationalguard_O["Side"] = "1"
VO_df_nona_bluelivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("Blue Lives Matter", case = False)]
VO_df_nona_bluelivesmatter_O["Side"] = "1"
VO_df_nona_htbluelivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("#BlueLivesMatter", case = False)]
VO_df_nona_htbluelivesmatter_O["Side"] = "1"
VO_df_nona_alllivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("All Lives Matter", case = False)]
VO_df_nona_alllivesmatter_O["Side"] = "1"
VO_df_nona_htalllivesmatter_O = VO_df_nona[VO_df_nona['O_string'].str.contains("#AllLivesMatter", case = False)]
VO_df_nona_htalllivesmatter_O["Side"] = "1"
VO_df_nona_police_O = VO_df_nona[VO_df_nona['O_string'].str.contains("police", case = False)]
VO_df_nona_police_O["Side"] = "1"
VO_df_nona_government_O = VO_df_nona[VO_df_nona['O_string'].str.contains("government", case = False)]
VO_df_nona_government_O["Side"] = "1"

VO_df_nona_terms = pd.concat([VO_df_nona_breonna_O,
                               VO_df_nona_floyd_O,
                               VO_df_nona_trayvon_O,
                               VO_df_nona_movement_O,
                               VO_df_nona_blacklivesmatter_O,
                               VO_df_nona_blm_O,
                               VO_df_nona_htblacklivesmatter_O,
                               VO_df_nona_protester_O,
                               VO_df_nona_demonstrator_O,
                               VO_df_nona_nationalguard_O,
                               VO_df_nona_bluelivesmatter_O,
                               VO_df_nona_htbluelivesmatter_O,
                               VO_df_nona_alllivesmatter_O,
                               VO_df_nona_htalllivesmatter_O,
                               VO_df_nona_police_O,
                               VO_df_nona_government_O
                              ],
                                 ignore_index=True)

VO_df_nona_terms

<ipython-input-5-fc976a512bcc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VO_df_nona_breonna_O["Side"] = "-1"
<ipython-input-5-fc976a512bcc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VO_df_nona_floyd_O["Side"] = "-1"
<ipython-input-5-fc976a512bcc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

,ID,Date,News_source_name,VOs,V,O,V_string,O_string,Side
0,31964104,20-07-18,vulture.com,"([mentions], [Breonna, Taylor])",[mentions],"[Breonna, Taylor]",mentions,Breonna Taylor,-1
1,31972010,20-07-23,washingtonpost.com,"([murdered], [Breonna, Taylor])",[murdered],"[Breonna, Taylor]",murdered,Breonna Taylor,-1
2,31874644,20-06-01,nymag.com,"([killed], [Breonna, Taylor])",[killed],"[Breonna, Taylor]",killed,Breonna Taylor,-1
3,31882645,20-06-05,chicago.suntimes.com,"([named], [[Breonna, Taylor], [Johnson]])",[named],"[[Breonna, Taylor], [Johnson]]",named,[Breonna Taylor] [Johnson],-1
4,31883040,20-06-05,euronews.com,"([shot], [Breonna, Taylor])",[shot],"[Breonna, Taylor]",shot,Breonna Taylor,-1
...,...,...,...,...,...,...,...,...,...
9586,85238290,20-06-22,Reason,"([devised], [Government])",[devised],[Government],devised,Government,1
9587,85255498,20-06-24,Reason,"([supposed], [government])",[supposed],[government],supposed,government,1
9588,85255498,20-06-24,Reason,"([identify], [government])",[identify],[government],identify,government,1
9589,85269390,20-06-25,NaN,"([accused], [government])",[accused],[government],accused,government,1


## Lemmatizing the V columns to enable the merger with connotation score dataset, and to allow for later analysis ##

In [6]:
# running on filtered term list dfs from above
# following this guide: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/
# import these modules

from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

def lemmatizer_fun(verb):
    verb_lemmatized = lemmatizer.lemmatize(verb, 'v')
    return verb_lemmatized

SV_df_nona_terms['V_lemmatized'] = SV_df_nona_terms.apply(lambda row: lemmatizer_fun(row["V_string"]), axis = 1)
VO_df_nona_terms['V_lemmatized'] = VO_df_nona_terms.apply(lambda row: lemmatizer_fun(row["V_string"]), axis = 1)

## Connotation dataset ##

In [7]:
# connotation scores originating from this paper: https://aclanthology.org/P16-1030.pdf

connotation_df = pd.read_csv("data/connotation_dataset/full_frame_info.txt", sep='\t')
connotation_df

,verb,Perspective(wo),Perspective(ws),Perspective(so),Effect(o),Effect(s),Value(o),Value(s),State(o),State(s),Perspective(ro),Perspective(rs),Perspective(os)
0,have,0.366667,0.333333,0.466667,0.066667,0.200000,0.466667,0.600000,0.066667,0.366667,0.400000,0.366667,0.066667
1,say,0.000000,0.166667,0.066667,0.133333,0.066667,0.066667,1.000000,0.000000,0.100000,0.033333,0.266667,0.000000
2,make,-0.066667,0.100000,0.100000,0.466667,-0.066667,0.200000,1.000000,0.000000,0.166667,-0.033333,0.200000,0.000000
3,take,0.500000,0.600000,0.800000,0.133333,0.600000,0.466667,0.600000,0.033333,0.666667,0.500000,0.566667,0.066667
4,go,0.066667,0.133333,0.200000,0.000000,0.066667,-0.066667,0.466667,-0.100000,0.033333,0.066667,0.066667,-0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,breed,-0.366667,-0.500000,0.000000,0.266667,-0.066667,0.066667,0.066667,-0.166667,-0.100000,-0.400000,-0.500000,0.000000
943,enrich,0.566667,0.533333,0.300000,0.866667,0.266667,1.000000,0.600000,0.433333,0.400000,0.466667,0.433333,0.533333
944,court,0.266667,0.433333,0.366667,0.400000,0.400000,0.866667,1.000000,0.333333,0.466667,0.333333,0.366667,0.300000
945,reconcile,0.400000,0.566667,0.133333,0.533333,0.333333,0.600000,0.466667,0.400000,0.033333,0.566667,0.600000,0.433333


In [8]:
# lemmatizing connotation score df to make sure it can be matched to the SV, VO dataframes by verb

# following this guide: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/
# import these modules

from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

def lemmatizer_fun(verb):
    verb_lemmatized = lemmatizer.lemmatize(verb, 'v')
    return verb_lemmatized

connotation_df['V_lemmatized'] = connotation_df.apply(lambda row: lemmatizer_fun(row["verb"]), axis = 1)

connotation_df

,verb,Perspective(wo),Perspective(ws),Perspective(so),Effect(o),Effect(s),Value(o),Value(s),State(o),State(s),Perspective(ro),Perspective(rs),Perspective(os),V_lemmatized
0,have,0.366667,0.333333,0.466667,0.066667,0.200000,0.466667,0.600000,0.066667,0.366667,0.400000,0.366667,0.066667,have
1,say,0.000000,0.166667,0.066667,0.133333,0.066667,0.066667,1.000000,0.000000,0.100000,0.033333,0.266667,0.000000,say
2,make,-0.066667,0.100000,0.100000,0.466667,-0.066667,0.200000,1.000000,0.000000,0.166667,-0.033333,0.200000,0.000000,make
3,take,0.500000,0.600000,0.800000,0.133333,0.600000,0.466667,0.600000,0.033333,0.666667,0.500000,0.566667,0.066667,take
4,go,0.066667,0.133333,0.200000,0.000000,0.066667,-0.066667,0.466667,-0.100000,0.033333,0.066667,0.066667,-0.100000,go
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,breed,-0.366667,-0.500000,0.000000,0.266667,-0.066667,0.066667,0.066667,-0.166667,-0.100000,-0.400000,-0.500000,0.000000,breed
943,enrich,0.566667,0.533333,0.300000,0.866667,0.266667,1.000000,0.600000,0.433333,0.400000,0.466667,0.433333,0.533333,enrich
944,court,0.266667,0.433333,0.366667,0.400000,0.400000,0.866667,1.000000,0.333333,0.466667,0.333333,0.366667,0.300000,court
945,reconcile,0.400000,0.566667,0.133333,0.533333,0.333333,0.600000,0.466667,0.400000,0.033333,0.566667,0.600000,0.433333,reconcile


## Merging filtered SV, VO dfs with connotation frame scores

In [9]:
# running on filtered term list dfs from above

SV_df_nona_terms_connotation = SV_df_nona_terms.merge(connotation_df, on = 'V_lemmatized', how = 'left')

VO_df_nona_terms_connotation = VO_df_nona_terms.merge(connotation_df, on = 'V_lemmatized', how = 'left')

In [10]:
# running on filtered term list dfs from above

# removing rows that have no connotation rating for the verb

print(SV_df_nona_terms_connotation.verb.isna().sum())
print(VO_df_nona_terms_connotation.verb.isna().sum())

term_list_df_SV_connotation_nonaverb = SV_df_nona_terms_connotation[SV_df_nona_terms_connotation['verb'].notna()]
term_list_df_VO_connotation_nonaverb = VO_df_nona_terms_connotation[VO_df_nona_terms_connotation['verb'].notna()]

print(term_list_df_SV_connotation_nonaverb.verb.isna().sum())
print(term_list_df_VO_connotation_nonaverb.verb.isna().sum())

4396
1790
0
0


## News media bias sources

In [11]:
# reading in media bias df

media_bias_df = pd.read_csv("data/news_political_alignment_ratings/news_sources_matching_bestversion.csv", sep = ';')

media_bias_df

,name_in_NOW_data,name_in_allsides_data,media_bias_rating,accessed,column,url,url_allsides,category
0,Axios,Axios,Center,from AllSides December 2021,NaN,https://www.axios.com/,https://www.allsides.com/news-source/axios,News Media
1,Barron's,NaN,Center,Ad Fontes Media Bias Chart 2021,NaN,NaN,NaN,NaN
2,Business Insider,Business Insider,Center,from supervisor 2019,allsides,NaN,NaN,NaN
3,Business Insider on MSN.com,Business Insider,Center,from supervisor 2019,allsides,NaN,NaN,NaN
4,businessinsider.com,Business Insider,Center,from supervisor 2019,allsides,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
160,The Raw Story,Raw Story,Left,from AllSides December 2021,NaN,http://www.rawstory.com/,https://www.allsides.com/news-source/raw-story,News Media
161,Fox News,Fox News (Online News),Right,from AllSides December 2021,NaN,http://www.foxnews.com/,https://www.allsides.com/news-source/fox-news-...,News Media
162,foxnews.com,Fox News (Online News),Right,from AllSides December 2021,NaN,http://www.foxnews.com/,https://www.allsides.com/news-source/fox-news-...,News Media
163,ksl.com,KSL,Right,from AllSides December 2021,NaN,http://www.ksl.com/,https://www.allsides.com/news-source/ksl,News Media


In [12]:
# renaming SV, VO dfs news source column to enable left join merger

term_list_df_SV_connotation_nonaverb = term_list_df_SV_connotation_nonaverb.rename(columns={'News_source_name': 'name_in_NOW_data'})
term_list_df_VO_connotation_nonaverb = term_list_df_VO_connotation_nonaverb.rename(columns={'News_source_name': 'name_in_NOW_data'})

print(len(term_list_df_SV_connotation_nonaverb))
print(len(term_list_df_VO_connotation_nonaverb))

30159
7811


In [13]:
# left joining media bias df to the SV, VO dataframes

SV_connotation_bias_df = term_list_df_SV_connotation_nonaverb.merge(media_bias_df, on = 'name_in_NOW_data', how = 'left')
VO_connotation_bias_df = term_list_df_VO_connotation_nonaverb.merge(media_bias_df, on = 'name_in_NOW_data', how = 'left')

SV_connotation_bias_df.to_csv("data/NOW_preprocessed/SV_BLM_bias_connotation_combined.csv", index = False)
VO_connotation_bias_df.to_csv("data/NOW_preprocessed/VO_BLM_bias_connotation_combined.csv", index = False)